In [5]:
import os

import numpy as np
import pandas as pd
import yaml


In [3]:
class CFG:
    n_folds = 5
    group_key = "series_id"

    filename = "/kaggle/input/targetdownsample_train_series_hour.parquet"
    output_filename = "/kaggle/input/targetdownsample_train_series_skffold.parquet"
    fold_yamls = [
        f"/kaggle/input/fold_split_yaml/stfk_event_count_bins105_fold{i}.yaml"
        for i in range(n_folds)
    ]


In [4]:
series_df = pd.read_parquet(CFG.filename)


In [13]:
event_df = pd.read_csv("/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv")


In [6]:
series_df["fold"] = -1
fold = 0
fold_yaml_path = CFG.fold_yamls[fold]
with open(fold_yaml_path, "r") as f:
    fold_series_dict = yaml.load(f, Loader=yaml.FullLoader)
print(len(fold_series_dict["train_series_ids"]),len(fold_series_dict["valid_series_ids"]),)


221 56


In [19]:
series_unique = series_df["series_id"].unique()
fold_valid_series = fold_series_dict["valid_series_ids"]
print(len(series_unique), len(fold_valid_series))
and_series = set(series_unique) & set(fold_valid_series)
print(and_series)
print(len(and_series))

not_include_series = list(set(fold_valid_series) - set(series_unique))
print(not_include_series)


269 56
{'a81f4472c637', 'ce9164297046', '3d53bfea61d6', 'fcca183903b7', '1319a1935f48', '137b99e936ab', 'df33ae359fb5', '83fa182bec3a', 'd9e887091a5c', '31011ade7c0a', 'd8de352c2657', 'e2a849d283c0', '3a9a9dc2cbd9', 'cca14d1966c1', 'b84960841a75', '1716cd4163b2', '6a4cd123bd69', 'a9a2f7fac455', 'db5e0ee1c0ab', '3aceb17ef7bd', 'a261bc4b7470', 'e6ddbaaf0639', '44d8c02b369e', '29c75c018220', '0ec9fc461819', '29d3469bd15d', '99237ce045e4', '40dce6018935', '91127c2b0e60', '207eded97727', '8898e6db816d', 'd5e47b94477e', '927dd0c35dfd', '76237b9406d5', 'ce85771a714c', '8877a6586606', 'a2b0a64ec9cf', 'a88088855de5', '1087d7b0ff2e', '3be2f86c3e45', '4b45c36f8f5a', '78569a801a38', '03d92c9f6f8a', '5aad18e7ce64', '67f5fc60e494', 'd25e479ecbb7', 'def21f50dd3c', '483d6545417f', '51b23d177971', '9c91c546e095', 'd2d6b9af0553', 'f56824b503a0', '33ceeba8918a', '72bbd1ac3edf'}
54
['390b487231ce', 'a3e59c2ce3f6']


In [22]:
df = event_df[event_df["series_id"].isin(not_include_series)]
print(len(df))
df.isnull().sum()


96


series_id     0
night         0
event         0
step         96
timestamp    96
dtype: int64

In [24]:
series_df.loc[series_df["series_id"].isin(fold_series_dict["valid_series_ids"]), "fold"] = fold
print(len(series_df[series_df["fold"] == fold]))


16274160


In [26]:
series_df["fold"] = -1
for fold, fold_yaml_path in enumerate(CFG.fold_yamls):
    with open(fold_yaml_path, "r") as f:
        fold_series_dict = yaml.load(f, Loader=yaml.FullLoader)
    print(
        len(fold_series_dict["train_series_ids"]),
        len(fold_series_dict["valid_series_ids"]),
    )
    series_df.loc[series_df["series_id"].isin(fold_series_dict["valid_series_ids"]), "fold"] = fold
    print(len(series_df[series_df["fold"] == fold]["series_id"].unique()))

    fold_valid_series = fold_series_dict["valid_series_ids"]
    series_unique = series_df["series_id"].unique()
    not_include_series = list(set(fold_valid_series) - set(series_unique))
    print(not_include_series)
    df = event_df[event_df["series_id"].isin(not_include_series)]
    print(len(df))
    display(df.isnull().sum())


221 56
54
['390b487231ce', 'a3e59c2ce3f6']
96


series_id     0
night         0
event         0
step         96
timestamp    96
dtype: int64

221 56
55
['e11b9d69f856']
46


series_id     0
night         0
event         0
step         46
timestamp    46
dtype: int64

222 55
54
['c5d08fc3e040']
46


series_id     0
night         0
event         0
step         46
timestamp    46
dtype: int64

222 55
53
['c7b1283bb7eb', '2fc653ca75c7']
92


series_id     0
night         0
event         0
step         92
timestamp    92
dtype: int64

222 55
53
['0f9e60a8e56d', '89c7daa72eee']
74


series_id     0
night         0
event         0
step         74
timestamp    74
dtype: int64